# First-Class Functions

Functions in Python are first-class objects.

* Created at runtime
* Assigned to a variable or element in a data structure
* Passed as an argument to a function
* Returned as the result of a function

## Treating a Function Like an Object

In [3]:
def factorial(n):
    '''returns n!'''
    return 1 if n < 2 else n * factorial(n-1)

In [4]:
print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
returns n!
<class 'function'>


We can assign a function object to a variable and pass it to a function as an argument.

In [6]:
fact = factorial
print(fact)

print(fact(5))

list(map(factorial, range(11)))

<function factorial at 0x7efc5defd6a8>
120


[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## Higher-Order Functions

A function that takes a function as argument or returns a function as the result is a _higher-order function_.

In [7]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

### Modern Replacements for map, filter, and reduce

`map` and `filter` are not important due to the introduction of list comprehensions and generator expressions.

In [9]:
print(list(map(fact, range(6))))
print(list(map(factorial, filter(lambda n: n % 2, range(6)))))

[1, 1, 2, 6, 24, 120]
[1, 6, 120]


reduce in Python 3 is in `functools` module. Common reducing built-ins
* sum
* all
* any

In [10]:
from functools import reduce
from operator import add
reduce(add, range(100))

4950

### Anonymous Functions

`lambda` creates an anonymous function.

In [11]:
sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## The Seven Flavors of Callable Objects

* User-defined functions. `def` or `lambda`
* Built-in functions
* Built-in methods
* Methods
* Classes: When invoked, a class runs its `__new__` method to create an instance, then `__init__` to initialize it and finally the instance is returned to the caller.
* Class instances: If a class defines a `__call__` method, then its instances may be invoked as functions.
* Generator functions. Functions or methods that use the `yield` keyword.

In [12]:
# use the callable to determine whether an object is callable
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

## User-Defined Callable Types

In [13]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        return self.pick()

In [14]:
bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())
print(callable(bingo))

1
0
True
